# Predicting students final exam outcome

This section aims to predict the `student final exam outcome`
(Pass (score >= 40) / Fail (score < 40)).
We try to replicate the machine learning analysis techinques from the work of
Tomasevic et al. (2020) {cite}`tomasevic_2020`.

**Keywords**: Predicting student outcome

```{bibliography}
:filter: docname in docnames
```

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from multicons import MultiCons

from oulad import get_oulad

oulad = get_oulad()

## Preparing train/test data

### Selecting features

In the work of Tomasevic et al. the student data from the `DDD` module of the
`2013J` and `2014B` presentations combined is used.

Similarly, we try to select the same seven distinct attributes from the three distinct
types below:

<table class="colwidths-auto table">
    <thead>
        <tr>
            <th class="text-center head"><p>DEMOGRAPHIC</p></th>
            <th class="text-center head"><p>ENGAGEMENT</p></th>
            <th class="text-center head"><p>PERFORMANCE</p></th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td class="text-left">
                <ul>
                    <li>gender</li>
                    <li>highest_education</li>
                    <li>age_band</li>
                </ul>
            </td>
            <td class="text-left">
                <ul>
                    <li>sum of clicks per assessment</li>
                </ul>
            </td>
            <td class="text-left">
                <ul>
                    <li>scores per assessment</li>
                    <li>number of attempts</li>
                    <li>final_exam score</li>
                </ul>
            </td>
        </tr>
    </tbody>
</table>

In [2]:
def get_feature_table(max_date=500, code_presentation="2013J"):
    """Returns the feature table computed from the OULAD dataset."""

    # Select all assessments from the module.
    assessments = oulad.assessments[
        (oulad.assessments.code_module == "DDD")
        & (oulad.assessments.code_presentation == code_presentation)
    ]

    # Filter out assessments that are after the max_date
    assessments = assessments[
        (assessments.date <= max_date) | (assessments.assessment_type == "Exam")
    ]

    # Filter relevant rows and columns from the student_vle table.
    vle = oulad.student_vle.loc[
        (oulad.student_vle.code_module == "DDD")
        & (oulad.student_vle.code_presentation == code_presentation),
        ["id_student", "date", "sum_click"],
    ]

    # Categorize the date field by assessment date.
    previous_date = None
    for date in assessments.date:
        if previous_date:
            vle.loc[(vle.date > previous_date) & (vle.date < date), "date"] = date
        else:
            vle.loc[vle.date < date, "date"] = date
        previous_date = date

    # Sum scores by date.
    vle = vle.groupby(["id_student", "date"]).agg(np.sum).reset_index()

    # Reshape the vle table.
    vle = vle.pivot(index="id_student", columns="date", values="sum_click")

    # Rename columns
    vle = vle.rename(
        columns={
            assessment.date: f"assessment_{i+1}_sum_click"
            if assessment.assessment_type != "Exam"
            else "final_exam_sum_click"
            for i, (_, assessment) in enumerate(assessments.iterrows())
        }
    ).drop("final_exam_sum_click", axis=1)

    return (
        oulad.student_info.loc[
            (oulad.student_info.code_module == "DDD")
            & (oulad.student_info.code_presentation == code_presentation),
            [
                "id_student",
                "gender",
                "highest_education",
                "age_band",
                "num_of_prev_attempts",
                # The `final_result` column is only used to fill missing `final_exam`
                # values, it should be removed from the training set.
                "final_result",
            ],
        ]
        .set_index("id_student")
        .join(vle)
        .join(
            oulad.student_assessment[
                oulad.student_assessment.id_assessment.isin(assessments.id_assessment)
            ]
            .pivot(index="id_student", columns="id_assessment", values="score")
            .rename(
                columns={
                    assessment.id_assessment: f"assessment_{i+1}_score"
                    if assessment.assessment_type != "Exam"
                    else "final_exam_score"
                    for i, (_, assessment) in enumerate(assessments.iterrows())
                }
            )
        )
    )


feature_table = pd.concat(
    [get_feature_table(), get_feature_table(code_presentation="2014B")]
)
display(feature_table)

,gender,highest_education,age_band,num_of_prev_attempts,final_result,assessment_1_sum_click,assessment_2_sum_click,assessment_3_sum_click,assessment_4_sum_click,assessment_5_sum_click,assessment_6_sum_click,assessment_1_score,assessment_2_score,assessment_3_score,assessment_4_score,assessment_5_score,assessment_6_score,final_exam_score
id_student,,,,,,,,,,,,,,,,,,
3733,M,HE Qualification,55<=,0,Withdrawn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8462,M,HE Qualification,55<=,0,Withdrawn,340.0,176.0,90.0,40.0,NaN,NaN,93.0,83.0,87.0,NaN,NaN,NaN,NaN
27417,M,Lower Than A Level,0-35,0,Withdrawn,43.0,180.0,103.0,23.0,52.0,9.0,48.0,58.0,52.0,6.0,NaN,NaN,NaN
27935,F,Lower Than A Level,0-35,0,Fail,301.0,286.0,61.0,118.0,47.0,NaN,75.0,73.0,90.0,63.0,NaN,NaN,NaN
28046,F,HE Qualification,35-55,0,Fail,127.0,131.0,20.0,45.0,202.0,53.0,58.0,57.0,49.0,49.0,69.0,24.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692948,F,Lower Than A Level,35-55,1,Pass,2.0,28.0,107.0,59.0,47.0,113.0,NaN,NaN,73.0,50.0,53.0,41.0,53.0
2694886,M,HE Qualification,0-35,0,Pass,210.0,125.0,43.0,82.0,224.0,45.0,71.0,76.0,85.0,66.0,64.0,62.0,69.0
2696376,F,Lower Than A Level,0-35,3,Fail,NaN,40.0,87.0,1.0,53.0,92.0,NaN,56.0,39.0,NaN,27.0,26.0,NaN


### Pre-Processing

#### Handling NAs

We notice many missing values from the `final_exam_score` column in the selected
feature table.

In [3]:
print(
    f"The feature table has {len(feature_table)} rows and the final exam score "
    f"column has {feature_table.final_exam_score.isna().sum()} rows with NAs "
    f"({100*feature_table.final_exam_score.isna().sum() / len(feature_table):.0f}%)."
)

The feature table has 3166 rows and the final exam score column has 1674 rows with NAs (53%).


This is explained in the original OULAD paper of Kuzilek et al.
[\[KHZ17\]](../notebooks/first_descriptive_analysis.html#id1):
```
Results of the final exam are usually missing (since they are scored and used for the
final marking immediately at the end of the module).
```

Therefore we use the `final_results` column to fill the missing final exam
values and then remove the `final_results` column.

Other columns containing missing values we fill with the value `-1`.

In [4]:
def fill_nas(feature_table_df):
    """Fills NAs in the `final_exam_score` column with `final_result` values,
    drops the `final_result` column and fills remaining NAs with the value `-1`.
    """

    final_exam_score_nas = feature_table.final_exam_score.isna()
    feature_table_df.loc[final_exam_score_nas, "final_exam_score"] = (
        feature_table[final_exam_score_nas].final_result.isin(["Pass", "Distinction"])
        * 40
    )
    return feature_table.drop(columns="final_result").fillna(-1)


feature_table = fill_nas(feature_table)
display(feature_table)

,gender,highest_education,age_band,num_of_prev_attempts,assessment_1_sum_click,assessment_2_sum_click,assessment_3_sum_click,assessment_4_sum_click,assessment_5_sum_click,assessment_6_sum_click,assessment_1_score,assessment_2_score,assessment_3_score,assessment_4_score,assessment_5_score,assessment_6_score,final_exam_score
id_student,,,,,,,,,,,,,,,,,
3733,M,HE Qualification,55<=,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
8462,M,HE Qualification,55<=,0,340.0,176.0,90.0,40.0,-1.0,-1.0,93.0,83.0,87.0,-1.0,-1.0,-1.0,0.0
27417,M,Lower Than A Level,0-35,0,43.0,180.0,103.0,23.0,52.0,9.0,48.0,58.0,52.0,6.0,-1.0,-1.0,0.0
27935,F,Lower Than A Level,0-35,0,301.0,286.0,61.0,118.0,47.0,-1.0,75.0,73.0,90.0,63.0,-1.0,-1.0,0.0
28046,F,HE Qualification,35-55,0,127.0,131.0,20.0,45.0,202.0,53.0,58.0,57.0,49.0,49.0,69.0,24.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692948,F,Lower Than A Level,35-55,1,2.0,28.0,107.0,59.0,47.0,113.0,-1.0,-1.0,73.0,50.0,53.0,41.0,53.0
2694886,M,HE Qualification,0-35,0,210.0,125.0,43.0,82.0,224.0,45.0,71.0,76.0,85.0,66.0,64.0,62.0,69.0
2696376,F,Lower Than A Level,0-35,3,-1.0,40.0,87.0,1.0,53.0,92.0,-1.0,56.0,39.0,-1.0,27.0,26.0,0.0


#### Splitting train / test data and Normalization

Now we randomly split the feature table rows into a train (80%) and test (20%) table
and, as in the work of Tomasevic et al., we scale and nomalize the selected features:

<table class="colwidths-auto table">
    <thead>
        <tr>
            <th class="text-center head"><p>Feature</p></th>
            <th class="text-center head"><p>Normalization</p></th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td class="text-left">
                <ul>
                    <li>gender</li>
                </ul>
            </td>
            <td class="text-left">
                <ul>
                    <li>0 = male</li>
                    <li>1 = female</li>
                </ul>
            </td>
        </tr>
        <tr>
            <td class="text-left">
                <ul>
                    <li>age_band</li>
                </ul>
            </td>
            <td class="text-left">
                <ul>
                    <li>0.0 = 0-35</li>
                    <li>0.5 = 35-55</li>
                    <li>1.0 = 55&lt;=</li>
                </ul>
            </td>
        </tr>
        <tr>
            <td class="text-left">
                <ul>
                    <li>highest_education</li>
                </ul>
            </td>
            <td class="text-left">
                <ul>
                    <li>0.00 = No Formal quals</li>
                    <li>0.25 = Lower Than A Level</li>
                    <li>0.50 = A Level or Equivalent</li>
                    <li>0.75 = HE Qualification</li>
                    <li>1.00 = Post Graduate Qualification</li>
                </ul>
            </td>
        </tr>
        <tr>
            <td class="text-left">
                <ul>
                    <li>number of attempts</li>
                    <li>sum of clicks per assessment</li>
                </ul>
            </td>
            <td class="text-left">
                <ul>
                    <li>0-N scaled to [0-1]</li>
                </ul>
            </td>
        </tr>
        <tr>
            <td class="text-left">
                <ul>
                    <li>scores per assessment</li>
                    <li>final_exam_score</li>
                </ul>
            </td>
            <td class="text-left">
                <ul>
                    <li>0-100 scaled to [0-1]</li>
                </ul>
            </td>
        </tr>
    </tbody>
</table>

In [10]:
RANDOM_STATE = 0


def normalized_train_test_split(feature_table_df, is_for_classification=True):
    """Returns the normalized tain/test split computed form the feature table.

    If `is_for_classification` is set to true (default) the final_exam_score will be
    converted into two classes 0 (score < 40 == Fail) and 1 (score >= 40 == Pass).
    """

    x_train_, x_test_, y_train_, y_test_ = train_test_split(
        feature_table_df.drop(columns="final_exam_score"),
        feature_table_df["final_exam_score"],
        test_size=0.2,
        random_state=RANDOM_STATE,
    )

    # Scale scores per assessment and final_exam_score.
    assessment_score_labels = feature_table_df.columns.values[
        feature_table_df.columns.str.match(r"assessment_[0-9]+_score")
    ]
    x_train_.loc[:, assessment_score_labels] /= 100
    x_test_.loc[:, assessment_score_labels] /= 100
    y_train_ /= 100
    y_test_ /= 100
    if is_for_classification:
        y_train_ = (y_train_ >= 0.4).astype(int)
        y_test_ = (y_test_ >= 0.4).astype(int)

    # Transform gender, age_band and highest_education to numeric values.
    gender_map = {"M": 0, "F": 1}
    age_band_map = {"0-35": 0, "35-55": 0.5, "55<=": 1}
    highest_education_map = {
        "No Formal quals": 0,
        "Lower Than A Level": 0.25,
        "A Level or Equivalent": 0.5,
        "HE Qualification": 0.75,
        "Post Graduate Qualification": 1,
    }

    x_train_.loc[:, "gender"] = x_train_.loc[:, "gender"].map(gender_map)
    x_test_.loc[:, "gender"] = x_test_.loc[:, "gender"].map(gender_map)

    x_train_.loc[:, "age_band"] = x_train_.loc[:, "age_band"].map(age_band_map)
    x_test_.loc[:, "age_band"] = x_test_.loc[:, "age_band"].map(age_band_map)

    x_train_.loc[:, "highest_education"] = x_train_.loc[:, "highest_education"].map(
        highest_education_map
    )
    x_test_.loc[:, "highest_education"] = x_test_.loc[:, "highest_education"].map(
        highest_education_map
    )

    # Scale sum of click per assessment and number of attempts.
    columns_slice = feature_table_df.columns.values[
        feature_table_df.columns.str.match(r"assessment_[0-9]+_sum_click")
    ].tolist() + ["num_of_prev_attempts"]

    # Note: we fit the scaler only on the train data to avoid "leaking" information
    # from the test data.
    scaler = MinMaxScaler().fit(x_train_.loc[:, columns_slice])
    x_train_.loc[:, columns_slice] = scaler.transform(x_train_.loc[:, columns_slice])
    x_test_.loc[:, columns_slice] = scaler.transform(x_test_.loc[:, columns_slice])
    return (x_train_, x_test_, y_train_, y_test_)


x_train, x_test, y_train, y_test = normalized_train_test_split(feature_table)
display(x_train)

,gender,highest_education,age_band,num_of_prev_attempts,assessment_1_sum_click,assessment_2_sum_click,assessment_3_sum_click,assessment_4_sum_click,assessment_5_sum_click,assessment_6_sum_click,assessment_1_score,assessment_2_score,assessment_3_score,assessment_4_score,assessment_5_score,assessment_6_score
id_student,,,,,,,,,,,,,,,,
546040,0,0.75,0.0,0.000000,0.016715,0.000564,0.063941,0.026418,0.008721,0.026269,0.61,0.58,0.77,0.66,0.51,0.57
391537,0,0.5,0.0,0.333333,0.027163,0.006484,0.116352,0.035742,0.021076,0.089102,0.34,0.36,0.39,0.36,0.30,0.75
571682,0,0.5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01
376280,1,0.5,0.5,0.166667,0.017133,0.000000,0.000000,0.000000,0.000000,0.000000,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01
577638,1,0.5,0.0,0.000000,0.081488,0.012405,0.049266,0.023310,0.006541,0.002485,0.75,0.77,0.55,-0.01,-0.01,-0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537747,1,0.75,0.5,0.000000,0.142499,0.039752,0.177149,0.035742,0.123910,0.036564,0.89,0.87,0.92,0.80,0.71,0.69
552747,1,0.25,0.5,0.000000,0.485583,0.042430,0.208595,0.310023,0.332485,0.173589,0.90,0.92,0.97,0.97,0.94,0.92
608848,1,0.5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01


## Classification

As in the work of Tomasevic et al., we will compare the classification performances
for the student final exam pass prediction (score >= 40).

We use the same models and try to perform a grid search over the same Hyper-parameter
ranges if these were specified in the paper:

- K-Nearest Neighbours (with & without `weights`, varying `K` between 1 and 50)
- Support Vector Machines (with `linear` and `RBF` kernels, varying `C` in
`[0.1, 1.0, 10]`, varying gamma in `[0.0001, 0.01, 0.1]`)
- Artificial Neural Networks (with one and two hidden layers)
- Decision Trees (with varying `max depth`, `split` strategy and `quality measure`)
- Naïve Bayes (with varying `var_smoothing`)
- Logistic Regression (with `lbfgs` and `saga` solvers)

And the performance metric used here is also the F1 score.

As a reminder, the formula of the F1 score is:
2 * (precision * recall) / (precision + recall)

However, in contrast to the paper, we use 5-fold cross validation during the grid
search phase.

In [16]:
# Hyperparameter search space

classifier_hyperparameters = {
    # K-Nearest Neighbours
    KNeighborsClassifier: [
        # {"n_neighbors": range(1, 51), "weights":["uniform", "distance"]}
        # We reduce search space for speed
        {
            "n_neighbors": [24],
            "weights": ["distance"],
        }
    ],
    # Support Vector Machines
    SVC: [
        # {
        #     "kernel": ["linear"],
        #     "C": [0.1, 1.0, 10],
        #     "probability": [True],
        #     "random_state": [RANDOM_STATE],
        # },
        {
            "kernel": ["rbf"],
            "C": [10],  # [0.1, 1.0, 10],
            "gamma": ["scale"],  # ["scale", "auto", 0.0001, 0.01, 0.1],
            "probability": [True],
            "random_state": [RANDOM_STATE],
        },
    ],
    # Artificial Neural Networks
    MLPClassifier: [
        {
            "max_iter": [1000],
            "validation_fraction": [0.2],
            "hidden_layer_sizes": [(10,)],  # [(10,), (20,), (52, 10)],
            "random_state": [RANDOM_STATE],
            # [(i,) for i in range(2, 100, 10)] + [
            #     (i, j) for i in range(2, 100, 10) for j in range(2, 100, 10)
            # ],
            # As we do not notice any improvement by varying `activation` and `alpha`,
            # we choose to keep the default values for these parameters.
            # "activation": ["logistic", "tanh", "relu"],
            # "alpha": 10.0 ** (- np.arange(-1,6))
        },
    ],
    # Decision Tree
    DecisionTreeClassifier: [
        {
            "criterion": ["entropy"],  # ["gini", "entropy"],
            "splitter": ["best"],  # ["best", "random"],
            "max_depth": [6],  # [None, *list(range(1, 11))],
            "min_samples_split": [2],  # range(2, 11, 2),
            "min_samples_leaf": [10],  # range(2, 11, 2),
            "random_state": [RANDOM_STATE],
        },
    ],
    # Naive Bayes
    GaussianNB: [
        {
            "var_smoothing": [1e-9],  # [1e-9, 1e-8, 1e-7, 1e-6]
        }
    ],
    # Logistic Regression
    LogisticRegression: [
        {
            "solver": ["lbfgs"],  # ["lbfgs", "saga"],
            "random_state": [RANDOM_STATE],
        }
    ],
}

for classifier, hyperparameters in classifier_hyperparameters.items():
    gs_classifier = GridSearchCV(classifier(), hyperparameters, scoring="f1", n_jobs=-1)
    gs_classifier.fit(x_train, y_train)
    print(
        f"{classifier.__name__}: score={gs_classifier.score(x_test, y_test):.4f} "
        f"best_parameters={gs_classifier.best_params_}"
    )

KNeighborsClassifier: score=0.8962 best_parameters={'n_neighbors': 24, 'weights': 'distance'}
SVC: score=0.8950 best_parameters={'C': 10, 'gamma': 'scale', 'kernel': 'rbf', 'probability': True, 'random_state': 0}
MLPClassifier: score=0.8914 best_parameters={'hidden_layer_sizes': (10,), 'max_iter': 1000, 'random_state': 0, 'validation_fraction': 0.2}
DecisionTreeClassifier: score=0.8719 best_parameters={'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 10, 'min_samples_split': 2, 'random_state': 0, 'splitter': 'best'}
GaussianNB: score=0.8806 best_parameters={'var_smoothing': 1e-09}
LogisticRegression: score=0.8919 best_parameters={'random_state': 0, 'solver': 'lbfgs'}


### Classification at different points in time

Predicting student final exam outcome seems to be more valuable at an early stage of
the course as it might give instuctors more time to help the students at risk.
However, predicting early is more challenging as less data is available for the
classifiers.

As in the work of Tomasevic et al., we will compare the classification performances at
different moments of the course based on the number of assessments passed.

Let's start by taking a look at the assessment table for the selected courses.

In [17]:
oulad.assessments[
    (oulad.assessments.code_module == "DDD")
    & (oulad.assessments.assessment_type == "TMA")
    & (
        (oulad.assessments.code_presentation == "2013J")
        | (oulad.assessments.code_presentation == "2014B")
    )
].sort_values("date")

,code_module,code_presentation,id_assessment,assessment_type,date,weight
88,DDD,2013J,25348,TMA,25.0,10.0
95,DDD,2014B,25355,TMA,25.0,10.0
89,DDD,2013J,25349,TMA,53.0,12.5
96,DDD,2014B,25356,TMA,53.0,12.5
97,DDD,2014B,25357,TMA,74.0,17.5
90,DDD,2013J,25350,TMA,88.0,17.5
98,DDD,2014B,25358,TMA,116.0,20.0
91,DDD,2013J,25351,TMA,123.0,20.0
99,DDD,2014B,25359,TMA,158.0,20.0
92,DDD,2013J,25352,TMA,165.0,20.0


We note that each course module has six intermediary assessments.

Next, we use the final submisssion `date` field to filter out assessment related
information after a given date and repeat the same data preprocessing and
classification process as done previously.

We also add Voting and MultiCons ensemble methods to check whether they might improve
current results.

In [18]:
result = {}
# We select the date such as both courses include the same amount of assessments
# after the filter.
for day in [25, 53, 88, 123, 165, 207]:
    result[day] = []
    feature_table = pd.concat(
        [
            get_feature_table(day),
            get_feature_table(day, code_presentation="2014B"),
        ]
    )
    feature_table = fill_nas(feature_table)
    x_train, x_test, y_train, y_test = normalized_train_test_split(feature_table)
    print(f"Computing classification results at day={day:.0f}...")

    train_predictions = []
    predictions = []
    estimators = []
    for classifier, hyperparameters in classifier_hyperparameters.items():
        gs_classifier = GridSearchCV(
            classifier(), hyperparameters, scoring="f1", n_jobs=-1
        )
        gs_classifier.fit(x_train, y_train)
        estimators.append((classifier.__name__, gs_classifier))
        predictions.append(gs_classifier.predict(x_test))
        train_predictions.append(gs_classifier.predict(x_train))
        result[day].append(round(f1_score(y_test, predictions[-1]), 4))

    # Voting Classifier
    voting = VotingClassifier(estimators=estimators, voting="soft")
    voting.fit(x_train, y_train)
    result[day].append(round(f1_score(y_test, voting.predict(x_test)), 4))

    # MultiCons
    multicons_options = {
        "similarity_measure": "JaccardIndex",
        "optimize_label_names": True,
        "consensus_function": "consensus_function_12",
    }
    # Searching for the best merging_threshold.
    max_score = 0  # pylint: disable=invalid-name
    merging_threshold = -1  # pylint: disable=invalid-name
    for mt in np.arange(0, 1, 0.05):
        recommended_consensus = (
            MultiCons(**multicons_options, merging_threshold=mt)
            .fit(train_predictions)
            .labels_.astype(bool)
        )
        score = f1_score(y_train, recommended_consensus)
        if score > max_score:
            max_score = score
            merging_threshold = mt
    print(f"MultiCons: selected merging_threshold={merging_threshold:0.2f}")

    recommended_consensus = (
        MultiCons(**multicons_options, merging_threshold=merging_threshold)
        .fit(predictions)
        .labels_.astype(bool)
    )
    result[day].append(round(f1_score(y_test, recommended_consensus), 4))


classifier_names = [
    classifier.__name__ for classifier in classifier_hyperparameters
] + ["Voting", "MultiCons"]

result_df = pd.DataFrame(result, index=classifier_names)
print("\nF1 score at different points in time:")
display(result_df)

Computing classification results at day=25...
MultiCons: selected merging_threshold=0.00
Computing classification results at day=53...
MultiCons: selected merging_threshold=0.00
Computing classification results at day=88...
MultiCons: selected merging_threshold=0.00
Computing classification results at day=123...
MultiCons: selected merging_threshold=0.05
Computing classification results at day=165...
MultiCons: selected merging_threshold=0.05
Computing classification results at day=207...
MultiCons: selected merging_threshold=0.70

F1 score at different points in time:


,25,53,88,123,165,207
KNeighborsClassifier,0.7038,0.7759,0.8035,0.8556,0.8787,0.8962
SVC,0.7513,0.7944,0.8193,0.8551,0.8771,0.8950
MLPClassifier,0.7266,0.7815,0.8267,0.8512,0.8809,0.8914
DecisionTreeClassifier,0.7034,0.7952,0.8179,0.8431,0.8771,0.8719
GaussianNB,0.7240,0.7873,0.8142,0.8456,0.8566,0.8806
LogisticRegression,0.7319,0.7842,0.8246,0.8539,0.8755,0.8919
Voting,0.7314,0.7851,0.8295,0.8619,0.8773,0.9013
MultiCons,0.7373,0.7819,0.8098,0.8487,0.8717,0.8789
